In [2]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated, List
import operator
import os

print("Successfully imported the neccessaries")

Successfully imported the neccessaries


In [3]:
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Load your key")

print("API key loaded")

API key loaded


In [4]:
llm =  ChatOpenAI(
    model = "gpt-5-nano",
    temperature = 0,
    api_key = api_key
)
print(f"{llm.model_name} Model has been  defined")

gpt-5-nano Model has been  defined


In [5]:
# creating the hybrid state

class HybridState(TypedDict):
    """"Combined state for plan_execute and Reflection pattern"""

    input: str
    plan: list[str]
    current_step: int
    results: Annotated[List[str], operator.add]
    #reflection
    draft: str
    critique: str
    iterations: int
    final_output: str

MAX_REFLECTION = 2
print("Defined the hybrid state!")



Defined the hybrid state!


In [7]:
# lets define the nodes
# planner node
def planner(state: HybridState) -> dict:
    """Create a step-by-step plan for the task"""
    prompt = f"""Create a step-by-step plan for the task:

Task: {state['input']}
Return a numbered list of concrete steps. Keep it concise(3-5 steps)."""
    response = llm.invoke([HumanMessage(content=prompt)])
    # paerse the steps
    lines = response.content.split('\n')
    steps = [line.strip() for line in lines if line.strip() and any(char.isdigit() for char in line[:3])]

    print(f"\n Plan has been created:")
    for step in steps:
        print(f"  {step}")
    print()
    
    return {"plan": steps, "current_step": 0, "results": []}

# executor node
def executor(state: HybridState) -> dict:
    """Execute each of the plans"""
    if state["current_step"] >= len(state["plan"]):
        #All steps has been executed
        return {}
    
    current_step = state["plan"][state["current_step"]]

    print(f"Executing step: {current_step}")

    #execute the step
    prompt = f"""Previous results: {state.get('results', [])}\n\nExecute this step: {current_step}"""
    response = llm.invoke([HumanMessage(content=prompt)])

    result= f"Step: {state['current_step'] + 1} result: {response.content}"
    print(result)

    return {
        "results": [result],
        "current_step": state["current_step"] + 1,
    }

# generator node

def generator(state: HybridState) -> dict:
    """Generate a draft for the result from the executor"""
    prompt = f"""Create a clear and complete answer to the result from the executor:

Original_query: {state['input']}
Executor_result: {state['results']}"""

    print("Generating initial draft...")
    response = llm.invoke([HumanMessage(content=prompt)])
    return {
        "draft": response.content, "iteration": 0
    }

#critic node
def critic(state: HybridState) -> dict:
    """Evaluate the draft for improvements"""
    prompt = f"""Improve the draft ensuring criterias like quality, completenesss and accuracy are met:
Original_query: {state['input']}
Draft: {state['draft']} 
Critique the response. What could be improved?
If it's excellent, say "APPROVED: explanation".
Otherwise, provide specific improvements needed"""

    print("Criticizing draft...")
    response = llm.invoke([HumanMessage(content=prompt)])
    return {
        "critique": response.content, "iteration": state["iteration"] + 1
    }

#refiner node

def refiner(state: HybridState) -> dict:
    """Refine the draft based on the critique"""
    if state["critique"] == "APPROVED":
        return {"final_draft": state["draft"]}
    
    #improve the draft
    prompt = f"""Improve the draft based on the critique:
Original_query: {state['input']}
Current_draft: {state['draft']} 
Critique: {state['critique']}"""
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"draft": response.content, "iteration": state["iteration"] + 1}

#finalizer node
def finalizer(state: HybridState) -> dict:
    """Create the final draft"""
    print("Reflection completed!")
    return {"final_draft": state["draft"]}


print("All nodes has been created successfully!")






All nodes has been created successfully!


In [8]:
# routing functions

def re_execute(state: HybridState) -> Literal["executor", "finalizer"]:
    """Decide if more steps to execute."""
    if state["current_step"] < len(state["plan"]):
        return "executor"
    return "finalizer"

def re_reflect(state: HybridState) -> Literal["generator", "finalizer"]:
    """Decide if we need more refinement."""
    # Stop if approved or max iterations
    if "APPROVED" in state.get("critique", "").upper():
        return "finalizer"
    
    if state["iterations"] >= MAX_REFLECTIONS:
        print(f"⚠️ Max iterations ({MAX_REFLECTIONS}) reached\n")
        return "finalizer"
    
    return "generator"

print("Router functions created successfully")

Router functions created successfully


In [9]:
#lets build the graph
builder = StateGraph(HybridState)

builder.add_node("planner", planner)
builder.add_node("executor", executor)
builder.add_node("generator", generator)
builder.add_node("critic", critic)
builder.add_node("refiner", refiner)
builder.add_node("finalizer", finalizer)

builder.add_edge(START, "planner")
builder.add_edge("planner", "executor")
builder.add_conditional_edges(
    "executor",
    re_execute,
    {"executor": "executor", "generator": "generator"}
)
builder.add_edge("generator", "critic")
builder.add_conditional_edges(
    "critic",
    re_reflect,
    {"refiner": "refiner", "finalizer": "finalizer"}
)
builder.add_edge("refiner", "critic") # Loop back for critiquing refinement
builder.add_edge("finalizer", END)

hybrid_agent= builder.compile()

print("Successfully built the hybrid agent")



Successfully built the hybrid agent


In [ ]:
#Lets test the hybrid agent out
result = hybrid_agent.invoke({
    "input": "Reasearch the benefits of Python Programming, create a summary and make it beginner-friendly",
    "plan": '',
    "current-step": 0,
    "results": [],
    "draft": "",
    "critique": "",
    "iterations": 0,
    "final_output": ""
})

print(f"\n{'='*70}")
print("📊 FINAL OUTPUT (after reflection):")
print(f"{'='*70}")
print(result["final_output"])
print(f"\nTotal iterations: {result['iterations']}")
print(f"{'='*70}\n")


 Plan has been created:
  1. Define scope and audience: decide to highlight beginner-friendly benefits (readability, versatility, libraries, community, job demand, rapid development) and set the tone for simple language.
  2. Gather sources and extract core benefits: collect 5–7 credible references (official Python site, beginner tutorials, respected guides) and pull concise benefits with plain-language examples.
  3. Draft the summary: write a short, beginner-friendly list (6–8 benefits) with one-sentence explanations and a brief example for each.
  4. Review and finalize: simplify wording, test with a non-expert reader, and add a tiny learning-path tip (how to start) at the end.

Executing step: 1. Define scope and audience: decide to highlight beginner-friendly benefits (readability, versatility, libraries, community, job demand, rapid development) and set the tone for simple language.
Step: 1 result: Here is a clear definition of scope and audience for this step.

Scope (what we a